In [ ]:
%pip install matplotlib
%pip install numpy
%pip install pandas

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 16 16:56:40 2022
ST2195 Question 1 
@author: celestlee

What is the best time of day / day of the week / time of year to fly to minimise delays?

"""

import os
os.getcwd()
os.chdir("/Users/celestlee/Desktop/data_files/dataverse_files")

from math import log10, floor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Preparing dataframe 
year_2005 = pd.read_csv("2005.csv.bz2")
year_2006 = pd.read_csv("2006.csv.bz2")
years = pd.concat([year_2005, year_2006])

years
years.info()

# Creating new column in dataframe 
def status(s):
    if s['DepDelay'] > 0:
        return 'Delayed'
    elif s['Diverted'] == 1:
        return 'Diverted'
    elif s['Cancelled'] == 1:
        return 'Cancelled'
    elif s['DepDelay'] <= 0 & s['Diverted'] == 0 & s['Cancelled'] == 0:
        return 'On Time'
years['status'] = years.apply(status, axis=1)
years['status'] = years['status'].astype('category')  

# Creating new dataframe with percentages of flight status
status_freq = years[["status"]].value_counts(sort=False).reset_index()
status_freq.rename(columns = {0: 'freq'}, inplace = True)
status_freq['perc'] = status_freq.freq / sum(status_freq.freq) * 100  
status_freq['perc'] = round(status_freq.perc, 2) 

    
# Bar plot of percentages of flight status
fig = plt.figure(figsize =(8, 5))
plt.bar(status_freq.status, status_freq.perc, width = 0.5)
plt.xlabel('Flight Status')
plt.ylim((0,65))
plt.ylabel('Percentage (%)')
plt.title('Overall Percentage of all flight status between Years 2005 and 2006') 

# Delay rate by month
month = years[['Month', 'status']].value_counts(sort=False).reset_index()
month.rename(columns = {0: 'freq'}, inplace = True)
month_num = month.groupby(by=['Month']).sum()
month = month[month['status'] == 'Delayed']
month = month.reset_index(drop=True)
month.insert(3, 'num_flights', month_num)
month['delay_rate'] = month.freq / month.num_flights
month['Month'] = month['Month'].replace({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May',
                        6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct',
                        11: 'Nov', 12: 'Dec'})


# Delay rate by day of month
day_month = years[['DayofMonth', 'status']].value_counts(sort=False).reset_index()
day_month.rename(columns = {0: 'freq'}, inplace = True)
day_month['DayofMonth'] = day_month['DayofMonth'].astype('category')
daymonth_num = day_month.groupby(by=['DayofMonth']).sum()
day_month = day_month[day_month['status'] == 'Delayed']
day_month = day_month.reset_index(drop=True)
day_month.insert(3, 'num_flights', daymonth_num)
day_month['delay_rate'] = day_month.freq / day_month.num_flights
day_month['DayofMonth'] = day_month['DayofMonth'].astype('category')

# Delay rate by day of week
week = years[['DayOfWeek', 'status']].value_counts(sort=False).reset_index()
week.rename(columns = {0: 'freq'}, inplace = True)
week_num = week.groupby(by=['DayOfWeek']).sum()
week = week[week['status'] == 'Delayed']
week = week.reset_index(drop=True)
week.insert(3, 'num_flights', week_num)
week['delay_rate'] = week.freq / week.num_flights
week['DayOfWeek'] = week['DayOfWeek'].replace({1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 
                                               5: 'Fri', 6: 'Sat', 7: 'Sun'}) 

# Delay rate by hour     
time = years.copy() ## Clean DepTime set hours as factors
time['DepTime'] = years['DepTime'].fillna(0).apply(np.int64) # Fills up NA's with 0
time = time[time['DepTime'] > 99] # Removes NA's
time = time[time['DepTime'] < 2401]
 
def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)
round_time = []
for i in time['DepTime']:
    if i > 99 and i < 1000:
        round_time.append(round_sig(i, 1))
    if i > 959 and i < 2401:
        round_time.append(round_sig(i, 2))
time['rounded_time'] = round_time 
time['rounded_time'] = time['rounded_time'].apply(lambda x: '{0:0>4}'.format(x))
time['rounded_time'] = time['rounded_time'].astype('category')

dephour = time[['rounded_time', 'status']].value_counts(sort=False).reset_index()
dephour.rename(columns = {0: 'freq'}, inplace = True)
dephour_num = dephour.groupby(by=['rounded_time']).sum()
dephour = dephour[dephour['status'] == 'Delayed']
dephour = dephour.reset_index(drop=True)
dephour.insert(3, 'num_flights', dephour_num)
dephour['delay_rate'] = dephour.freq / dephour.num_flights

# 2x2 plot
fig, ax = plt.subplots(2, 2, figsize = (15,10))
fig.suptitle("Delay Rates based on", fontsize=16)
ax[0][0].plot(month['Month'], month['delay_rate'])
ax[0][0].title.set_text('Month')

ax[0][1].plot(day_month['DayofMonth'], day_month['delay_rate'])
ax[0][1].title.set_text('Day of Month')
ax[0][1].set(xticks = day_month['DayofMonth'], xticklabels = day_month['DayofMonth'])

ax[1][0].plot(week['DayOfWeek'], week['delay_rate'])
ax[1][0].title.set_text('Day of Week')

ax[1][1].plot(dephour['rounded_time'], dephour['delay_rate'])
ax[1][1].title.set_text('Hour')
ax[1][1].set_xticklabels(dephour['rounded_time'], rotation = 90)
 
plt.show()


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 25 18:35:00 2022
ST2195 Question 2
@author: celestlee

Do older planes suffer more delays?

"""

import os
os.getcwd()
os.chdir("/Users/celestlee/Desktop/data_files/dataverse_files")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Preparing dataframe 
planes = pd.read_csv("plane-data.csv")
year_2005 = pd.read_csv("2005.csv.bz2")
year_2006 = pd.read_csv("2006.csv.bz2")
year_2007 = pd.read_csv("2007.csv.bz2")
years = pd.concat([year_2005, year_2006, year_2007])

# years
# years.info() 
# planes.info()
# planes.describe()
               
# Histogram of year of manufacture
planes = planes.dropna() # Remove NA's
plane_year = planes[["year", "tailnum"]]
plane_year = plane_year[plane_year['year'] != 'None'] 
plane_year = plane_year[plane_year['year'] != '0000']
plane_year = plane_year.sort_values(by = 'year', ascending = True)
plane_year['year'] = plane_year['year'].astype(int)

fig, ax = plt.subplots()
fig.suptitle("Year of Manufacture", fontsize=13)
p = plt.hist(plane_year['year'])
plt.xticks(rotation='vertical')
plt.xticks(np.arange(min(plane_year['year']), max(plane_year['year'])+1, 10.0))

plt.show()

# Preparing data for 2x1 line chart
years2 = years[['Year', 'DepDelay', 'ArrDelay', 'TailNum']]
years2.rename(columns = {'TailNum': 'tailnum'}, inplace = True)
plane_age = years2.merge(plane_year, on = 'tailnum', how = 'right')
plane_age = plane_age.dropna()
plane_age['year'] = plane_age['year'].astype(int)
plane_age['age'] = plane_age.Year - plane_age.year 
plane_age = plane_age[plane_age['age'] > 0]
plane_test = plane_age.groupby(['age']).mean().reset_index()
plane_test['age'] = plane_test['age'].astype(int)

# 2x1 Line chart of mean departure and arrival delay against years

fig, ax = plt.subplots()
plane_test.plot('age', 'DepDelay', ax = ax, alpha = 0.8)
plane_test.plot('age', 'ArrDelay', ax = ax, alpha = 0.8)
ax.legend(['Mean Departure Delay', 'Mean Arrival Delay'], loc = 'lower left')
ax.axvline(x = 25 , color='k', linestyle=':', alpha = 0.6)
ax.set_xlabel('  Years of Service')
ax.title.set_text("Mean Delay Rates based on Year of Manufacture (minutes)")

# Delay factors based on years (grouped bar plot)
years3 = years[['Year', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 
                'SecurityDelay', 'LateAircraftDelay', 'TailNum']]
years3.rename(columns = {'TailNum': 'tailnum'}, inplace = True)
delay_factors = years3.merge(plane_year, on = 'tailnum', how = 'right')
delay_factors = delay_factors.dropna()
delay_factors['year'] = delay_factors['year'].astype(int)
delay_factors['age'] = delay_factors.Year - delay_factors.year 
delay_factors = delay_factors[delay_factors['age'] > 0]
delay = delay_factors.groupby(['age']).mean().reset_index()
delay['group']=np.where((delay['age'] > 25),'Over 25 years', 'Below 25 years')
delay = delay[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 
               'LateAircraftDelay', 'group']]
delay['group'] = delay['group'].astype('category')
delay = delay.groupby(['group']).sum().reset_index()

below_delay = delay[delay['group'] == 'Below 25 years']
below_melt = pd.melt(below_delay, id_vars = 'group')
below_melt = below_melt[['variable', 'value']]
below_melt.rename(columns = {'value': 'Below 25 years'}, inplace = True)

over_delay = delay[delay['group'] == 'Over 25 years'] 
over_melt = pd.melt(over_delay, id_vars = 'group')
over_melt = over_melt[['value']] 
over_melt.rename(columns = {'value': 'Over 25 years'}, inplace = True)
delayed_group = pd.concat([below_melt, over_melt], axis=1)
variable = list(set(list(below_melt['variable']))) 
delayed_group.set_index('variable', inplace=True)
          
fig, ax = plt.subplots()
ax = delayed_group[['Below 25 years', 'Over 25 years']].plot.bar(alpha = 0.8)
ax.set_xlabel('Delay Factors')
ax.set_ylabel('Mean Delay (in minutes)')
plt.legend(fontsize = 7)
plt.show()

# Mean Carrier, NAS, LAteAircraft Delay against all years (Line chart) 3x1
years4 = years[['Year', 'CarrierDelay', 'NASDelay', 'LateAircraftDelay', 'TailNum']]             
years4.rename(columns = {'TailNum': 'tailnum'}, inplace = True)
main_factors = years4.merge(plane_year, on = 'tailnum', how = 'right')
main_factors = main_factors.dropna()
main_factors['year'] = main_factors['year'].astype(int)
main_factors['age'] = main_factors.Year - main_factors.year 
main_factors = main_factors[main_factors['age'] > 0]               
factors = main_factors.groupby(['age']).mean().reset_index()               
factors = factors[['age', 'CarrierDelay', 'NASDelay', 'LateAircraftDelay']]               

# 3x1 Line Chart 
fig, ax = plt.subplots(1, 3, figsize = (20,6))                
fig.suptitle("Mean Delay based on the Top 3 Delay Factors", fontsize=16, fontweight = 'bold')              
ax[0].plot(factors['age'], factors['CarrierDelay'])               
ax[0].title.set_text('Carrier Delay')  
ax[0].set_xlabel('  Years of service')    
ax[0].set_ylabel('Mean Delay (minutes)')   
ax[0].set_ylim([0, 7])        
               
ax[1].plot(factors['age'], factors['NASDelay'])               
ax[1].title.set_text('NAS Delay') 
ax[1].set_xlabel('  Years of service') 
ax[1].set_ylabel('Mean Delay (minutes)') 
ax[1].set_ylim([0, 7])     

ax[2].plot(factors['age'], factors['LateAircraftDelay'])               
ax[2].title.set_text('Late Aircraft Delay') 
ax[2].set_xlabel('  Years of service') 
ax[2].set_ylabel('Mean Delay (minutes)') 
ax[2].set_ylim([0, 7])     

plt.show()
               

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 27 00:23:21 2022
ST2195 Question 3
@author: celestlee

How does the number of people flying between different locations change over time?

"""

import os
os.getcwd()
os.chdir("/Users/celestlee/Desktop/data_files/dataverse_files")

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

# Preparing dataframe 
year_2005 = pd.read_csv("2005.csv.bz2")
year_2006 = pd.read_csv("2006.csv.bz2")
year_2007 = pd.read_csv("2007.csv.bz2")
years = pd.concat([year_2005, year_2006, year_2007])

years
years.info()

# Creating Origin-Dest Pairs
od_pairs = years[['Year', 'Origin', 'Dest']]
combi = pd.DataFrame(years.Origin + '/' + years.Dest).astype('category')
combi.rename(columns = {0: 'combi'}, inplace = True)
od_pairs = pd.concat([od_pairs, combi], axis = 1)

# For own reference to retrieve top 10 OD pairs
sum_od = od_pairs[['combi']].value_counts().reset_index()
sum_od.rename(columns = {0: 'freq'}, inplace = True)
sum_od = sum_od.sort_values(by = 'freq', ascending = False)
sum_od = sum_od.iloc[0:9,]

# Bar plot for own reference
fig, ax = plt.subplots(figsize = (9,7))
ax.barh(sum_od.combi, sum_od.freq)
ax.set_xlabel('Count')
ax.set_ylabel('Origin/Destination')
ax.title.set_text('Origin-Destination Combinations with the most trips')

# Top 10 OD pairs
top_od_pairs = od_pairs.loc[(od_pairs['combi'] == 'SAN/LAX') | (od_pairs['combi'] == 'OGG/HNL') |
                        (od_pairs['combi'] == 'LGA/DCA') | (od_pairs['combi'] == 'LGA/BOS') |
                        (od_pairs['combi'] == 'LAX/SAN') | (od_pairs['combi'] == 'LAX/LAS') | 
                        (od_pairs['combi'] == 'LAS/LAX') | (od_pairs['combi'] == 'HNL/OGG') |
                        (od_pairs['combi'] == 'DCA/LGA') | (od_pairs['combi'] == 'BOS/LGA')]

top_od_pairs = top_od_pairs[['combi', 'Year']].value_counts().reset_index()
top_od_pairs.rename(columns = {0: 'freq'}, inplace = True)
top_od_pairs = top_od_pairs.pivot_table(index = 'combi', columns = 'Year', 
                                        values = 'freq').sort_index()

ax = top_od_pairs.plot.barh(stacked=True, alpha = 0.9)
ax.set_xlabel('Number of Trips')
ax.set_ylabel('Origin/Destination')
ax.title.set_text('Top 10 Origin-Destination Combinations')
plt.legend(loc = 'upper left',fontsize = 8)
plt.show()

# Top 10 OD with significant changes in traffic

unusual_pairs = od_pairs[['Year', 'combi']]

unusual_pairs1 = unusual_pairs[unusual_pairs['Year'] == 2005]
unusual_pairs1 = pd.DataFrame(unusual_pairs1[["combi"]].value_counts().sort_index())
unusual_pairs1.rename(columns = {0: '2005'}, inplace = True)

unusual_pairs2 = unusual_pairs[unusual_pairs['Year'] == 2006]
unusual_pairs2 = pd.DataFrame(unusual_pairs2[["combi"]].value_counts().sort_index())
unusual_pairs2.rename(columns = {0: '2006'}, inplace = True)

unusual_pairs3 = unusual_pairs[unusual_pairs['Year'] == 2007]
unusual_pairs3 = pd.DataFrame(unusual_pairs3[["combi"]].value_counts().sort_index())
unusual_pairs3.rename(columns = {0: '2007'}, inplace = True)

odd_pairs = pd.concat([unusual_pairs1, unusual_pairs2, unusual_pairs3], axis=1)

odd_pairs['max'] = odd_pairs[['2005', '2006', '2007']].max(axis=1)
odd_pairs['min'] = odd_pairs[['2005', '2006', '2007']].min(axis=1)
odd_pairs['max_diff'] = odd_pairs['max'] - odd_pairs['min']
odd_pairs = odd_pairs.sort_values(by='max_diff', ascending=False)
odd_pairs = odd_pairs.iloc[:10]
odd_pairs = odd_pairs[['2005', '2006', '2007']]

ax1 = odd_pairs.plot.barh(stacked=True, alpha = 0.9)
ax1.set_xlabel('Number of Trips')
ax1.set_ylabel('Origin/Destination')
ax1.title.set_text('Origin-Destination Combinations with significant changes in traffic')
plt.legend(loc = 'upper right')
plt.show()

# Line chart of top 5 combinations
line_pairs = years[['Year', 'Month', 'DayofMonth', 'Origin', 'Dest']]
line_pairs = pd.DataFrame(line_pairs[['Year', 'Month', 'DayofMonth', 'Origin', 'Dest']].value_counts().reset_index())
line_pairs.rename(columns = {0: 'freq'}, inplace = True)
line_pairs = line_pairs.sort_values(by='freq', ascending=False)

line_pairs['combi'] = (line_pairs.Origin + '/' + line_pairs.Dest).astype('category')

top_5 = line_pairs.loc[(line_pairs['combi'] == 'SAN/LAX') | (line_pairs['combi'] == 'LAX/SAN') |
                      (line_pairs['combi'] == 'LAX/LAS') | (line_pairs['combi'] == 'LAS/LAX') |
                      (line_pairs['combi'] == 'BOS/LGA')]


top_5['date'] = ((top_5.Year).astype(str) + '-' + (top_5.Month).astype(str) + '-' + 
                 (top_5.DayofMonth).astype(str)).astype(str)

dates = top_5[['date']]
dates = pd.to_datetime(dates['date'])
dates = dates.dt.to_period('M')
dates = pd.DataFrame(dates)
top_5['date'] = dates['date']

mean_line_pairs = top_5[['combi', 'date', 'freq']]
mean_line_pairs = mean_line_pairs.pivot_table(mean_line_pairs, index=['combi', 'date'] ,aggfunc='mean').reset_index()
mean_line_pairs = mean_line_pairs.pivot_table(index = ['date'], columns = 'combi', values = 'freq').reset_index()
mean_line_pairs['date'] = mean_line_pairs['date'].dt.to_timestamp()

# Line chart of top 5 combinations with significant changes
change_5 = line_pairs.loc[(line_pairs['combi'] == 'OGG/HNL') | (line_pairs['combi'] == 'HNL/OGG') |
                      (line_pairs['combi'] == 'KOA/HNL') | (line_pairs['combi'] == 'HNL/LIH') |
                      (line_pairs['combi'] == 'HNL/KOA')]

change_5['date'] = ((change_5.Year).astype(str) + '-' + (change_5.Month).astype(str) + '-' + 
                 (change_5.DayofMonth).astype(str)).astype(str)

change_date = change_5[['date']]
change_date = pd.to_datetime(change_date['date'])
change_date = change_date.dt.to_period('M') 
change_date = pd.DataFrame(change_date)
change_5['date'] = change_date['date']

mean_change_pairs = change_5[['combi', 'date', 'freq']]
mean_change_pairs = mean_change_pairs.pivot_table(mean_change_pairs, index=['combi', 'date'] ,aggfunc='mean').reset_index()
mean_change_pairs = mean_change_pairs.pivot_table(index = ['date'], columns = 'combi', values = 'freq').reset_index()
mean_change_pairs['date'] = mean_change_pairs['date'].dt.to_timestamp()

# 2x1 Line chart
x = ['Jan 05', 'Jun 05', 'Dec 05', 'Jan 06', 'Jun 06', 'Dec 06', 'Jan 07', 'Jun 07', 'Dec 07', 'Jan 08']

fig, ax = plt.subplots(1,2,figsize = (18,8))
ax[0].plot(mean_line_pairs['date'], mean_line_pairs['BOS/LGA'], label = 'BOS/LGA')
ax[0].plot(mean_line_pairs['date'], mean_line_pairs['LAS/LAX'], label = 'LAS/LAX')
ax[0].plot(mean_line_pairs['date'], mean_line_pairs['LAX/LAS'], label = 'LAX/LAS')
ax[0].plot(mean_line_pairs['date'], mean_line_pairs['LAX/SAN'], label = 'LAX/SAN')
ax[0].plot(mean_line_pairs['date'], mean_line_pairs['SAN/LAX'], label = 'SAN/LAX')
ax[0].title.set_text('Traffic of Top 5 Origin-Destination Combinations over 3 years')  
ax[0].set_xlabel('Year') 
ax[0].set_ylabel('Mean Number of trips')
ax[0].legend()
ax[0].set_xticklabels(x)

ax[1].plot(mean_change_pairs['date'], mean_change_pairs['HNL/KOA'], label = 'HNL/KOA')
ax[1].plot(mean_change_pairs['date'], mean_change_pairs['HNL/LIH'], label = 'HNL/LIH')
ax[1].plot(mean_change_pairs['date'], mean_change_pairs['HNL/OGG'], label = 'HNL/OGG')
ax[1].plot(mean_change_pairs['date'], mean_change_pairs['KOA/HNL'], label = 'KOA/HNL')
ax[1].plot(mean_change_pairs['date'], mean_change_pairs['OGG/HNL'], label = 'OGG/HNL')
ax[1].title.set_text('Traffic of Origin-Destination Combinations with significant increases over 3 years')  
ax[1].set_xlabel('Year') 
ax[1].set_ylabel('Mean Number of trips')
ax[1].legend()
ax[1].set_xticklabels(x)

plt.show()

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 27 03:20:57 2022
ST2195 Question 4
@author: celestlee

Can you detect cascading failures as delays in one airport create delays in others?

"""

import os
os.getcwd()
os.chdir("/Users/celestlee/Desktop/data_files/dataverse_files")

from math import log10, floor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Preparing dataframe 
year_2005 = pd.read_csv("2005.csv.bz2")
year_2006 = pd.read_csv("2006.csv.bz2")
year_2007 = pd.read_csv("2007.csv.bz2")
years = pd.concat([year_2005, year_2006, year_2007])
#years = pd.concat([year_2005, year_2006])
years
years.info()

years['Month'] = years['Month'].astype('category')
years['DayofMonth'] = years['DayofMonth'].astype('category')                       
years['dep_delayed']=np.where((years['DepDelay'] > 0),'Yes', 'No')
years['arr_delayed']=np.where((years['ArrDelay'] > 0),'Yes', 'No')                           
years['dep_delayed'] = years['dep_delayed'].astype('category')
years['arr_delayed'] = years['arr_delayed'].astype('category')

def status(s):
    if s['DepDelay'] > 0:
        return 'Delayed'
    elif s['Diverted'] == 1:
        return 'Diverted'
    elif s['Cancelled'] == 1:
        return 'Cancelled'
    elif s['DepDelay'] <= 0 & s['Diverted'] == 0 & s['Cancelled'] == 0:
        return 'On Time'
years['status'] = years.apply(status, axis=1)
years['status'] = years['status'].astype('category')

# Delay rate per month plot (from Question 1)                           
month = years[['Month', 'status']].value_counts(sort=False).reset_index()
month.rename(columns = {0: 'freq'}, inplace = True)
month_num = month.groupby(by=['Month']).sum()
month = month[month['status'] == 'Delayed']
month = month.reset_index(drop=True)
month.insert(3, 'num_flights', month_num)
month['delay_rate'] = month.freq / month.num_flights
month['Month'] = month['Month'].replace({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May',
                        6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct',
                        11: 'Nov', 12: 'Dec'})

# Line chart of delay rate per month
fig, ax = plt.subplots()                          
ax.plot(month['Month'], month['delay_rate'])
ax.set_xlabel('Month')
ax.set_ylabel('Delay Rate')
ax.title.set_text('Delay Rate by Month') 
plt.show()
            
# Histogram for number of flights per month (grouped)
hist_month = years[['Month', 'Year']].value_counts(sort=False).reset_index()
hist_month.rename(columns = {0: 'freq'}, inplace = True)
hist_month = hist_month.pivot_table(index = 'Month', columns = 'Year', 
                                        values = 'freq')
hist_month = hist_month.rename(index={1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May',
                        6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct',
                        11: 'Nov', 12: 'Dec'})

ax = hist_month.plot.bar(stacked=False)
ax.set_xlabel('Month')
ax.set_ylabel('Frequency')
ax.title.set_text('Number of Flights')
plt.legend(fontsize = 8, loc = 'lower right')
plt.show()

# Month with highest delay rate
first_month = years[['Month', 'DayofMonth', 'dep_delayed', 'arr_delayed']]
first_month = first_month[first_month['Month'] == 12]
first_month = first_month[['DayofMonth', 'dep_delayed', 'arr_delayed']]
first_month = first_month.value_counts(sort=False).reset_index()
first_month.rename(columns = {0: 'freq'}, inplace = True)

dep_delay =	first_month[["DayofMonth", "dep_delayed", "freq"]]
dep_delay = dep_delay.pivot_table(index = 'DayofMonth', columns = 'dep_delayed', values = 'freq').sort_index()
dep_delay['dep_delay_rate'] = dep_delay.Yes / (dep_delay.Yes + dep_delay.No)
dep_delay = dep_delay[['dep_delay_rate']]

arr_delay =	first_month[["DayofMonth", "arr_delayed", "freq"]]
arr_delay = arr_delay.pivot_table(index = 'DayofMonth', columns = 'arr_delayed', values = 'freq').sort_index()
arr_delay['arr_delay_rate'] = arr_delay.Yes / (arr_delay.Yes + arr_delay.No)
arr_delay = arr_delay[['arr_delay_rate']]

delay = pd.concat([dep_delay, arr_delay], axis=1)
delay['Day'] = delay.index

# Day Delay
day = years[['Month', 'DayofMonth', 'DepTime', 'CRSDepTime', 'DepDelay', 'dep_delayed', 'arr_delayed']]
day = day[(day['Month'] == 12)]
day = day[(day['DayofMonth'] == 22)]
day['DepTime'] = day['DepTime'].fillna(0).apply(np.int64)
day = day.loc[(day.DepTime > 99) & (day.DepTime < 2401)]

def round_sig(x, sig=2):
    return round(x, sig-int(floor(log10(abs(x))))-1)

round_time = []

for i in day['DepTime']:
    if i > 99 and i < 1000:
        round_time.append(round_sig(i, 1))
    if i > 959 and i < 2401:
        round_time.append(round_sig(i, 2))
day['rounded_time'] = round_time
day['rounded_time'] = day['rounded_time'].apply(lambda x: '{0:0>4}'.format(x))
day['rounded_time'] = day['rounded_time'].astype('category')

day = day[['DayofMonth', 'rounded_time', 'dep_delayed', 'arr_delayed']]

num_dep = day[day['dep_delayed'] == 'Yes']
num_dep = len(num_dep)
day_dep = day[["DayofMonth", "rounded_time", "dep_delayed"]].value_counts().reset_index()
day_dep.rename(columns = {0: 'freq'}, inplace = True)
day_dep = day_dep.pivot_table(index = ['DayofMonth', 'rounded_time'], columns = 'dep_delayed', values = 'freq').sort_index()
day_dep['dep_delay_rate'] = day_dep.Yes / (day_dep.Yes + day_dep.No)

arr_dep = day[day['arr_delayed'] == 'Yes']
arr_dep = len(arr_dep)
arr_dep = day[["DayofMonth", "rounded_time", "arr_delayed"]].value_counts().reset_index()
arr_dep.rename(columns = {0: 'freq'}, inplace = True)
arr_dep = arr_dep.pivot_table(index = ['DayofMonth', 'rounded_time'], columns = 'arr_delayed', values = 'freq').sort_index()
arr_dep['arr_delay_rate'] = arr_dep.Yes / (arr_dep.Yes + arr_dep.No)

day = pd.concat([day_dep, arr_dep], axis=1)
day = day[['dep_delay_rate', 'arr_delay_rate']]
day.reset_index(inplace=True)

# Line Chart of analysis 1
fig, ax = plt.subplots(figsize = (20,7))
ax.plot(delay['Day'], delay['dep_delay_rate'])
ax.plot(delay['Day'], delay['arr_delay_rate'])
ax.title.set_text('Delay Rate in December')  

ax.set_xlabel('Day of Month') 
ax.set_ylabel('Delay Rate')
ax.set(xticks = delay['Day'], xticklabels = delay['Day'])
ax.legend()

# Line chart of analysis 2
fig, ax = plt.subplots(figsize = (20,7))
ax.plot(day['rounded_time'], day['dep_delay_rate'])
ax.plot(day['rounded_time'], day['arr_delay_rate'])
ax.set_xlabel('Hour')
ax.set_ylabel('Delay Rate')
ax.title.set_text('Delay Rate on 22nd December')

ax.legend()
plt.show()         

# Manually checking for cascading failures due to delays
check = years[['Year', 'Month', 'DayofMonth', 'CRSDepTime', 'DepTime', 'CRSArrTime', 'ArrTime',
               'TailNum', 'FlightNum', 'Origin', 'Dest', 'arr_delayed', 'dep_delayed']]
check = check[(check['Year'] == 2005) & (check['Month'] == 12) & (check['DayofMonth'] == 22) &
              (check['arr_delayed'] == 'Yes') & (check['dep_delayed'] == 'Yes') & 
              (check['DepTime'] > 1100) & (check['DepTime'] < 1300)]

# Found 2 examples of 1st degree cascading failures due to delays
check = check[(check['TailNum'] == 'N957SW') | (check['TailNum'] == 'N835AE')]
check


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 27 16:09:45 2022
ST2195 Question 5
@author: celestlee

"""
import os
os.getcwd()
os.chdir("/Users/celestlee/Desktop/data_files/dataverse_files")

from math import log10, floor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

planes = pd.read_csv("plane-data.csv")
year_2005 = pd.read_csv("2005.csv.bz2")
year_2006 = pd.read_csv("2006.csv.bz2")
year_2007 = pd.read_csv("2007.csv.bz2")
years = pd.concat([year_2005, year_2006, year_2007])
#years = year_2005
#years
#years.info()
#planes.info()
#planes.describe()

# Creating new column in dataframe 
def status(s):
    if s['DepDelay'] > 0:
        return 'Delayed'
    elif s['Diverted'] == 1:
        return 'Diverted'
    elif s['Cancelled'] == 1:
        return 'Cancelled'
    elif s['DepDelay'] <= 0 & s['Diverted'] == 0 & s['Cancelled'] == 0:
        return 'On Time'
years['status'] = years.apply(status, axis=1)
years['status'] = years['status'].astype('category')  


main = years[['Year', 'CRSDepTime', 'DepDelay', 'CRSArrTime', 'CRSElapsedTime',
              'status', 'TailNum']]
main = main.dropna()

# Checking percentage of unusual CRSDepTime
unusual_CRSDepTime = main[['CRSDepTime']]
num_unusual = len(unusual_CRSDepTime[unusual_CRSDepTime['CRSDepTime'] < 100]) # Number of 2 digits
num_usual = len(unusual_CRSDepTime[unusual_CRSDepTime['CRSDepTime'] > 99])
unusual_perc = pd.DataFrame([num_unusual, num_usual])
unusual_perc.rename(columns = {0: 'num'}, inplace = True)
unusual_perc['perc'] = unusual_perc.num / (num_usual + num_unusual) * 100
unusual_perc['category'] = ['Unusual CRSDepTime', 'Usual CRSDepTime'] 
unusual_perc = unusual_perc.set_index('category')

def round_sig(x, sig=2): # function that rounds off values to significant values 
    return round(x, sig-int(floor(log10(abs(x))))-1)

# Factor 1: CRSDepTime
crsdep = [] 

for i in main['CRSDepTime']:
    if  i >= 0 and i < 100: # minutes of 00:00  
        crsdep.append(2400)
    if i > 99 and i < 1000: 
        crsdep.append(round_sig(i, 1))
    if i > 959 and i < 2400:
        crsdep.append(round_sig(i, 2))
main['crsdep'] = crsdep
main['crsdep'] = main['crsdep'].apply(lambda x: '{0:0>4}'.format(x))
main['crsdep'] = main['crsdep'].astype('category')

crsdep = main[['crsdep', 'DepDelay']]
crsdep = crsdep.groupby(['crsdep']).mean().reset_index()

# Factor 2: CRSArrTime
crsarr = [] 

for i in main['CRSArrTime']:
    if  i >= 0 and i < 100: # minutes of 00:00  
        crsarr.append(2400)
    if i > 99 and i < 1000: 
        crsarr.append(round_sig(i, 1))
    if i > 959 and i < 2400:
        crsarr.append(round_sig(i, 2))
main['crsarr'] = crsarr
main['crsarr'] = main['crsarr'].apply(lambda x: '{0:0>4}'.format(x))
main['crsarr'] = main['crsarr'].astype('category')

crsarr = main[['crsarr', 'DepDelay']]
crsarr = crsarr.groupby(['crsarr']).mean().reset_index()

# 2x1 Line chart of `DepTime` and `CRSDepTime` against mean departure delay
fig, ax = plt.subplots()
crsdep.plot('crsdep', 'DepDelay', ax = ax, alpha = 0.8)
crsarr.plot('crsarr', 'DepDelay', ax = ax, alpha = 0.8)
ax.legend(['Departure Time', 'Scheduled Departure Time'], loc = 'lower left')
ax.axvline(x = 25 , color='k', linestyle=':', alpha = 0.6)
ax.set_xlabel('Hour')
ax.set_ylabel('Mean Departure Delay')
ax.title.set_text("Mean Departure Delay based on Departure Times")

# Factor 4: CRSElapsedTime
elapsed = main.iloc[0:99999,]
elapsed = elapsed.dropna()

ax = elapsed.plot.scatter('DepDelay', 'CRSElapsedTime', figsize = (8,8), alpha = 0.6)
ax.set_xlabel('Departure Delay')
ax.set_ylabel('Scheduled Elapsed Time')
ax.title.set_text("Scatter Plot of Scheduled Elapsed Time against Departure Delay (in minutes)")
plt.show()